这是TensorFlow的入门教程，展示了如何：

* 导入所需的包
* 创建和使用张量
* 使用GPU加速
* 演示 tf.data.Dataset

In [1]:
from __future__ import absolute_import, division, print_function

导入TensorFlow<br>
首先，导入tensorflow模块。从TensorFlow 2.0开始，默认情况下将打开急切执行功能。这样可以使TensorFlow更具交互性的前端，我们将在以后详细讨论其细节。

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.0.0'

张量<br>
张量是一个多维数组。与NumPy ndarray对象类似，tf.Tensor对象具有数据类型和形状。此外，tf.Tensors可以驻留在加速器内存（如GPU）中。TensorFlow提供了一个丰富的操作库（tf.add，tf.matmul，tf.linalg.inv等），这些操作消耗并产生tf.Tensors。这些操作会自动转换本机Python类型，例如：

In [4]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


每个tf.Tensor都有一个形状和一个数据类型：

In [5]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


In [6]:
type(x.numpy())

numpy.ndarray

NumPy数组和tf.Tensors 之间最明显的区别是：

1. 张量可以由加速器内存（如GPU，TPU）支持。
2. 张量是不变的。

NumPy兼容性
在TensorFlow tf.Tensor和NumPy 之间进行转换ndarray很容易：

* TensorFlow操作会自动将NumPy ndarray转换为Tensors。
* NumPy操作会自动将张量转换为NumPy ndarray。

使用其.numpy()方法将张量显式转换为NumPy ndarray 。这些转换通常很便宜，因为tf.Tensor如果可能的话，它们可以共享数组并共享底层的内存表示形式。但是，共享底层表示并不总是可能的，因为它们tf.Tensor可能托管在GPU内存中，而NumPy数组始终由主机内存支持，并且转换涉及从GPU到主机内存的复制。

In [7]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


GPU加速<br>
使用GPU进行计算可加速许多TensorFlow操作。没有任何注释，TensorFlow会自动决定是使用GPU还是CPU进行操作-如有必要，在CPU和GPU内存之间复制张量。由操作产生的张量通常由执行操作的设备的内存支持，例如：

In [8]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is the Tensor on GPU #0:  
True


设备名称<br>
该Tensor.device属性提供承载张量内容的设备的标准字符串名称。此名称编码许多详细信息，例如正在执行此程序的主机的网络地址以及该主机中的设备的标识符。这是分布式执行TensorFlow程序所必需的。GPU:<N>如果将张量放置在N主机的第-GPU上，则字符串以结尾。

显式设备放置<br>
在TensorFlow中，放置是指如何将单个操作分配（放置）在设备上以执行。如前所述，当没有提供明确的指导时，TensorFlow会自动决定执行哪个设备并在需要时将张量复制到该设备上。但是，可以使用tf.device上下文管理器将TensorFlow操作显式放置在特定设备上，例如：

In [12]:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)

    result = time.time()-start

    print("10 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

# Force execution on GPU #0 if available
if tf.config.experimental.list_physical_devices("GPU"):
    print("On GPU:")
    with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

On CPU:
10 loops: 90.88ms
On GPU:
10 loops: 0.00ms


数据集<br>
本部分使用tf.data.DatasetAPI来构建用于将数据输入模型的管道。该tf.data.DatasetAPI用于从简单，可重复使用的部分构建高性能，复杂的输入管道，这些输入管道将为模型的训练或评估循环提供支持。

建立来源 Dataset<br>
创建源使用的工厂实现相同的功能集Dataset.from_tensors，Dataset.from_tensor_slices或使用从类似文件中读取对象TextLineDataset或TFRecordDataset。有关更多信息，请参见TensorFlow数据集指南。

In [13]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

In [14]:
with open("test.cvs", 'w') as f:
    f.write("""Line 1
Line 2
Line 3
  """)

In [15]:
ds_file = tf.data.TextLineDataset("test.cvs")

应用转换<br>
使用如转换功能map，batch以及shuffle对转换应用到数据集的记录。

In [19]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [24]:
next(iter(ds_tensors))

<tf.Tensor: id=249, shape=(2,), dtype=int32, numpy=array([1, 4])>

重复<br>
tf.data.Dataset 对象支持迭代以循环记录：

In [27]:
print('Elements of ds_tensors:')
for x in ds_tensors:
    print(x)

print('\nElements in ds_file:')
for x in ds_file:
    print(x)

Elements of ds_tensors:
tf.Tensor([1 4], shape=(2,), dtype=int32)
tf.Tensor([ 9 16], shape=(2,), dtype=int32)
tf.Tensor([36 25], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)
